In [508]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from RNN_block import RNN_block
import pennylane as qml
import pandas as pd


In [509]:
df = pd.read_csv('./dataset.csv')

In [510]:
n_train = 250

y_train = torch.tensor(df['nat_demand'].to_numpy()[:n_train]).to(torch.float)
x_train = torch.tensor(df[sorted("T2M_toc	QV2M_toc TQL_toc W2M_toc T2M_san QV2M_san TQL_san W2M_san T2M_dav QV2M_dav TQL_dav W2M_dav".split())].to_numpy()[:n_train]).to(torch.float)

xs_train = torch.tensor([(x_train[:,3 * i : 3 * i + 3].sum(dim=1)/3).tolist() for i in range(4)]).T
print(xs_train[1])
y_train = (y_train-y_train.min())/(y_train.max()-y_train.min())




tensor([1.7476e-02, 2.3959e+01, 3.5131e-02, 1.2807e+01])


In [511]:
import pennylane as qml
class RNN_block:
    def __init__(self, input_size):
        self.input_size = input_size

    
    def embedding(self, params):
        n = (self.input_size + 1) // 2
        for i in range(n):
            qml.Hadamard(i)
            qml.RZ(2.0 * params[:,i], i)
        
        for i in range(n - 1):
            qml.IsingZZ(2.0 * params[:,n + i] ,[i , i + 1])
    
    def ansatz(self, params, all_entangled = False):
        # Length of Params : 3 * num_qubit
        n = self.input_size
        for i in range(n):
            qml.RX(params[3 * i], i)
            qml.RY(params[3 * i + 1], i)
            qml.RZ(params[3 * i + 2], i)
        for i in range(n - 1):
            qml.CNOT([i, i + 1])
        if all_entangled:
            qml.CNOT([n - 1, 0])

In [512]:
class G1(nn.Module):
    '''
    G for V(G)
    '''
    def __init__(self, input):
        super(G1, self).__init__()
        self.li1 = nn.Linear(input,16)
        self.li2 = nn.Linear(16, 16)
        self.li3 = nn.Linear(16, 2*input-1)
        
    def forward(self, x):
        x = self.li1(x)
        x = F.relu(x)
        x = self.li2(x)
        x = F.relu(x)
        x = self.li3(x)
        x = 2*torch.pi*F.relu(x)
        return x

In [513]:
n_qu = 4
dev = qml.device("default.qubit", wires = n_qu)

def embedding(params, n_qu):
    '''
    embedding layer
    '''
    n = n_qu
    for i in range(n):
        qml.Hadamard(i)
        qml.RZ(2.0 * params[:,i], i)
    
    for i in range(n - 1):
        qml.IsingZZ(2.0 * params[:,n + i] ,[i , i + 1])

@qml.qnode(dev, interface="torch")
def fidelity(vec1, vec2, n_qu):
    '''
        Args:
            vec1 : list, (2n - 1)개의 element로 이루어진 vector
            vec2 : list, (2n - 1)개의 element로 이루어진 vector
    '''
    embedding(vec1, n_qu) # Phi(x1) circuit 적용
    qml.adjoint(embedding)(vec2, n_qu) # Phi^t(x2) 적용
    return qml.probs()


In [514]:
from kan import KAN
from fucntions import data_seq,train_seq
class feature_train(nn.Module):
    def __init__(self,n_qu):
        super(feature_train, self).__init__()
        self.n_qu = n_qu
        self.linear1 = KAN([n_qu,n_qu*2+1,n_qu*2-1],grid=1)
        self.quantum_layer = fidelity
    def forward(self,inputs):
        input1 = inputs[0]
        input2 = inputs[1]
        input1 = self.linear1(input1)
        input2 = self.linear1(input2)
        output = self.quantum_layer(input1,input2,self.n_qu)[:,0]
        return output

        
                
        
                

In [515]:
train_list = []
train_label_list = []
test_list = []
test_label_list = []

for i in range(n_train-1):
    train_data = torch.stack([xs_train,torch.concat([xs_train[(i+1):],xs_train[:(i+1)]])])
    train_list.append(train_data)
    label_data = torch.stack([y_train,torch.concat([y_train[(i+1):],y_train[:(i+1)]])])
    train_label_list.append(label_data)

In [516]:
train_data = torch.concat(train_list,dim=1)
train_label = torch.concat(train_label_list,dim=1)

In [517]:
from torch import optim
pretrain_data = data_seq(train_data,train_label)
train_loader, test_loader = pretrain_data.split_data(batch_size=64,seq_first=True)
model = feature_train(n_qu)
optimizer = optim.Adam(model.parameters(),lr=0.005)

def criterion(pred,label):
    return -torch.sum(torch.log(1-pred)*(label[:,0]-label[:,1])**2 + torch.log(pred)*(1-(label[:,0]-label[:,1])**2))/len(pred)
    

In [507]:
pretrain_seq = train_seq(model,train_loader,test_loader)
pretrain_seq.train(3,optimizer,criterion,seq_first=True)

KeyboardInterrupt: 

In [493]:
block = RNN_block(4)
@qml.qnode(dev, interface="torch")
def ansatz(inputs,weights):
    '''
        Args:
            vec1 : list, (2n - 1)개의 element로 이루어진 vector
            vec2 : list, (2n - 1)개의 element로 이루어진 vector
    '''
    block.embedding(inputs)
    block.ansatz(weights) # Phi(x1) circuit 적용
    return qml.probs()

In [495]:
block = RNN_block(4)
class train_model(nn.Module):
    def __init__(self,feature_model):
        super(train_model, self).__init__()
        self.QNE_layer = feature_model.linear1
        self.ansatz = ansatz
        self.Q_param = nn.Parameter(torch.rand([12]).float(),requires_grad=True)
    def forward(self,input):
        input = self.QNE_layer(input)
        output = self.ansatz(input,self.Q_param)[:,0]
        return output
    
class train_model_fixed(nn.Module):
    def __init__(self):
        super(train_model_fixed, self).__init__()
        self.ansatz = ansatz
        self.Q_param = nn.Parameter(torch.rand([12]).float(),requires_grad=True)
    def forward(self,input):
        data_list = []
        feature_len = input.shape[1]
        for i in range(feature_len-1):
            data = (np.pi-input[:,i])*(np.pi-input[:,i+1])
            data_list.append(data)
        data = torch.stack(data_list,dim=1)
        input = torch.concat([input,data],dim=1)
            
        
        
        output = self.ansatz(input,self.Q_param)[:,0]
        return output
        

In [496]:
data = data_seq(xs_train,y_train)
train_loader,test_loader = data.split_data(batch_size=64)

In [499]:
CLS_model  = train_model(model)
optimizer = optim.Adam([CLS_model.Q_param],lr=0.04)
criterion = nn.MSELoss()

reg_seq = train_seq(CLS_model,train_loader,test_loader)

In [500]:
reg_seq.train(100,optimizer,criterion)

epoch : 1 loss :0.18464331328868866 loss_test = 0.20358189940452576
epoch : 2 loss :0.153375506401062 loss_test = 0.1592804491519928
epoch : 3 loss :0.11685360223054886 loss_test = 0.11393798887729645
epoch : 4 loss :0.08523280918598175 loss_test = 0.08010075241327286
epoch : 5 loss :0.06789699196815491 loss_test = 0.06528328359127045
epoch : 6 loss :0.06679517775774002 loss_test = 0.06347034871578217
epoch : 7 loss :0.07087789475917816 loss_test = 0.06357274204492569
epoch : 8 loss :0.06964418292045593 loss_test = 0.06408829987049103
epoch : 9 loss :0.06549665331840515 loss_test = 0.06908824294805527
epoch : 10 loss :0.06548474729061127 loss_test = 0.07465886324644089
epoch : 11 loss :0.06769930571317673 loss_test = 0.07623013854026794
epoch : 12 loss :0.06742370128631592 loss_test = 0.0711924359202385
epoch : 13 loss :0.0655263364315033 loss_test = 0.06798989325761795
epoch : 14 loss :0.06494953483343124 loss_test = 0.06538277119398117
epoch : 15 loss :0.06551557779312134 loss_test =

In [501]:
CLS_model  = train_model_fixed()
optimizer = optim.Adam([CLS_model.Q_param],lr=0.04)
criterion = nn.MSELoss()

reg_seq = train_seq(CLS_model,train_loader,test_loader)

In [502]:
reg_seq.train(100,optimizer,criterion)

epoch : 1 loss :0.22131291031837463 loss_test = 0.2660052180290222
epoch : 2 loss :0.2125205099582672 loss_test = 0.252135694026947
epoch : 3 loss :0.19893859326839447 loss_test = 0.23267321288585663
epoch : 4 loss :0.18025341629981995 loss_test = 0.20858950912952423
epoch : 5 loss :0.15985174477100372 loss_test = 0.18297429382801056
epoch : 6 loss :0.13945746421813965 loss_test = 0.15924471616744995
epoch : 7 loss :0.1214638501405716 loss_test = 0.13988447189331055
epoch : 8 loss :0.10846792906522751 loss_test = 0.12543658912181854
epoch : 9 loss :0.09766899049282074 loss_test = 0.1136385127902031
epoch : 10 loss :0.0886048972606659 loss_test = 0.10089411586523056
epoch : 11 loss :0.07941437512636185 loss_test = 0.08706020563840866
epoch : 12 loss :0.06996604055166245 loss_test = 0.07336292415857315
epoch : 13 loss :0.06227102875709534 loss_test = 0.06241871044039726
epoch : 14 loss :0.057357307523489 loss_test = 0.055275898426771164
epoch : 15 loss :0.05466345697641373 loss_test = 0.